In [0]:
%sql
DROP TABLE IF EXISTS stg_product; -- staging table to store 
CREATE TABLE IF NOT EXISTS stg_product (
  sku INT NOT NULL,
  product_name STRING,
  product_type STRING,
  description STRING,
  price DOUBLE
) USING DELTA;


### General constraint check

In [0]:
%sql
SELECT *
FROM silver.cleansed_sales
WHERE 
  product_type IS NULL
  OR total_revenue < 0                        
  OR sale_date IS NULL             

In [0]:
%sql
SELECT * 
FROM silver.cleansed_sales

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW viewProduct
AS
(
  SELECT
    sku,
    name,
    product_type,
    description,
    price
  FROM silver.cleansed_sales
)

In [0]:
spark.sql("INSERT INTO stg_product SELECT * FROM viewProduct")

### Create the `dim_product` table

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold.dim_product (
  sku INT NOT NULL,
  product_name STRING,
  product_type STRING,
  description STRING,
  price DOUBLE,
  starting_date DATE NOT NULL,
  ending_date DATE,
  activate BOOLEAN NOT NULL
) USING DELTA;


In [0]:
# df = spark.read.table("silver.cleansed_sales")

# # Show the data
# df.show()

In [0]:
# %sql 
# MERGE INTO gold.dim_product AS tgt -- target table 
# USING (
#   SELECT
#     sku,
#     name,
#     product_type,
#     description,
#     price,
#     sale_date AS start_date
#   FROM silver.cleansed_sales
# ) AS src
# ON tgt.sku = src.sku AND tgt.activate = true

# -- update changes
# WHEN MATCHED AND ( -- when the SKUs match
#        tgt.product_name != src.name   
#        OR tgt.product_type != src.product_type 
#        OR tgt.description != src.description
#        OR tgt.price != src.price
#      )
#   THEN UPDATE SET
#     tgt.ending_date = date_sub(src.start_date, 1), -- update the ending_date
#     tgt.activate  = false

# -- insert new versions for new SKUs
# WHEN NOT MATCHED AND (
#        tgt.activate = false
#        AND tgt.sku = src.sku
#      )
#   THEN INSERT (sku, product_name, product_type, description, price, starting_date, ending_date, activate)
#        VALUES (src.sku, src.name, src.product_type, src.description, src.price, src.start_date, NULL, true);


In [0]:
%sql
SELECT COUNT(*)
FROM gold.dim_product

In [0]:
%sql
UPDATE stg_product
SET name = "test update"
WHERE sku = 1000